<a href="https://colab.research.google.com/github/MThwa/crypto-analytics/blob/main/wallets_of_interest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import requests
import pandas as pd
from google.cloud import bigquery
from google.auth import default
import pandas_gbq
from datetime import datetime
from decimal import Decimal
import numpy as np

# Search Parameters

CHAINID = 8453
V2      = "https://api.etherscan.io/v2/api"

TOKENS = {
    "OPENLOOT_OL": "0xEEB221A559F4aD1746F41460807E56d06D198A58",
    "BIGTIME":     "0x3e1AC921d02Be0FCB88ad681f6c45E63e40C0927",
    "USDC":        "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913"
}

# Get our top depositors
credentials, project = default()
client = bigquery.Client(credentials=credentials, project=project)
data = client.query("""
  SELECT
    split(tf.urn_from,":")[3] AS urn_from,
    SUM(tf.amount) AS amount
  FROM `openloot-362008.postgres_rds_wallets_public.transactions` t
  JOIN `openloot-362008.postgres_rds_wallets_public.transfers` tf
    ON tf.transaction_id = t.id
  WHERE t.type='deposit'
    AND tf.currency = 'OL'
    AND t.created_at > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
  GROUP BY 1
  HAVING amount > 100000
  LIMIT 20
""").to_dataframe()

## added 20250915
deposits = client.query('''
  SELECT
  tf.urn_from,
  tf.amount,
  t.tx_hash
  FROM `openloot-362008.postgres_rds_wallets_public.transactions` t
  JOIN `openloot-362008.postgres_rds_wallets_public.transfers` tf ON tf.transaction_id = t.id
  WHERE t.type='deposit'
  AND tf.currency = 'OL'
  ORDER BY t.created_at DESC
''').to_dataframe()

# Helper Runctions
def is_contract_on_base(address: str) -> bool:
    """True if address has code on Base (i.e., a contract)."""
    params = {
        "chainid": CHAINID,
        "module": "proxy",
        "action": "eth_getCode",
        "address": address,
        "tag": "latest",
        "apikey": API_KEY,
    }
    r = requests.get(V2, params=params, timeout=30).json()
    # EVM returns "0x" for EOAs; anything else means contract
    return r.get("result", "0x") != "0x"

# get the token transactions
def fetch_tokentx(contract: str, address: str) -> list:
    """All ERC-20 transfers for one token+wallet on Base."""
    page, offset, rows = 1, 10000, []
    while True:
        params = {
            "chainid": CHAINID, "module": "account", "action": "tokentx",
            "address": address, "contractaddress": contract,
            "page": page, "offset": offset, "sort": "asc", "apikey": API_KEY,
        }
        r = requests.get(V2, params=params, timeout=30).json()
        if r.get("status") != "1":
            if r.get("result") in ("No transactions found", "No records found", [], None):
                break
            raise RuntimeError(r)
        batch = r.get("result", [])
        if not batch:
            break
        rows.extend(batch)
        if len(batch) < offset:
            break
        page += 1
        time.sleep(0.2)
    return rows


# --- build address list (EOAs only) ---
wallets = list(data["urn_from"].unique())
# keep EOAs (is_contract == False)
ADDRESSES = [w for w in wallets if not is_contract_on_base(w)]
# (optional) checksum/uppercase NOT required; avoid clobbering ADDRESSES

# --- main loop ---
all_rows = []
for address in ADDRESSES:
    for name, ca in TOKENS.items():
        txs = fetch_tokentx(ca, address)
        for r in txs:
            r["_wallet"] = address
            r["_token"] = name
        all_rows.extend(txs)

df = pd.DataFrame(all_rows)

###################
## Turn into flows
###################

df_lim = df[['timeStamp','hash','from','contractAddress','to','value',
             'tokenName','tokenSymbol','tokenDecimal','_wallet','_token']].copy()

# Types
df_lim['tokenDecimal'] = pd.to_numeric(df_lim['tokenDecimal'], errors='coerce')
df_lim['value'] = pd.to_numeric(df_lim['value'], errors='coerce')
df_lim['timeStamp'] = pd.to_datetime(df_lim['timeStamp'], unit='s')

# Token units
df_lim['amount_token'] = df_lim['value'] / (10.0 ** df_lim['tokenDecimal'])

# For each row, check whether the focal _wallet was sender or receiver
wl = df_lim['_wallet'].str.lower()
df_lim['direction'] = np.where(df_lim['to'].str.lower().eq(wl), 'in',
                        np.where(df_lim['from'].str.lower().eq(wl), 'out', 'other'))

# Keep only rows where the target wallet actually participated
df_lim = df_lim[df_lim['direction'] != 'other']

# Signed amount (relative to the target wallet only)
df_lim['amount_token'] = np.where(df_lim['direction'].eq('in'),
                                  df_lim['amount_token'],
                                  -df_lim['amount_token'])

# left join with deposit TXs
df_lim = df_lim.merge(deposits, left_on='hash', right_on='tx_hash', how='left')
df_lim['deposit_tx'] = np.where(df_lim['urn_from'].isnull(), 0, 1)
df_lim = df_lim.drop(columns=['urn_from', 'tx_hash','amount'])


# Final view
df_final = (
    df_lim.rename(columns={'timeStamp':'timestamp'})
          .assign(wallet=lambda d: d['_wallet'])
          [['timestamp','hash','contractAddress','tokenName','tokenSymbol','wallet','amount_token','deposit_tx']]
          .sort_values(['timestamp','hash'], kind='stable')
          .reset_index(drop=True)
)

# Mark wallets of interest
ADDRESSES_lower = {addr.lower() for addr in ADDRESSES}
df_final['wallet_of_interest'] = df_final['wallet'].str.lower().isin(ADDRESSES_lower)

# save the df
pandas_gbq.to_gbq(df_final, project_id='bigtimestudios', destination_table='currency_analytics.wallets_of_interest', if_exists='replace')
print('\nRuntime: ',datetime.now())



In [ ]:
OL_CONTRACT = "0xEEB221A559F4aD1746F41460807E56d06D198A58"

def get_balance(contract: str, wallet: str) -> float:
    """Return current ERC20 balance for wallet (in token units)."""
    params = {
        "chainid": CHAINID,
        "module": "account",
        "action": "tokenbalance",
        "contractaddress": contract,
        "address": wallet,
        "tag": "latest",
        "apikey": API_KEY,
    }
    r = requests.get(V2, params=params, timeout=30).json()
    if r.get("status") != "1":
        raise RuntimeError(f"Error fetching balance for {wallet}: {r}")
    raw = int(r["result"])  # integer in smallest unit
    # OL token uses 18 decimals
    return raw / (10 ** 18)

# check to make sure $OL balances aren't negative
df_check = df_final[(df_final['wallet_of_interest'] == True) & (df_final['tokenSymbol'] == 'OL')]
df_manual = df_check.groupby('wallet').agg({'amount_token':'sum'}).sort_values('amount_token',ascending=False).reset_index()

# Query balances for your ADDRESSES list
balances = []
for w in ADDRESSES:
    bal = get_balance(OL_CONTRACT, w)
    balances.append({"wallet": w, "OL_balance": bal})
    time.sleep(0.2)  # be nice to the API

df_balances = pd.DataFrame(balances)

pd.set_option('display.float_format', '{:.2f}'.format) # set to 2 digits
df_comp = pd.merge(df_manual, df_balances, on='wallet', how='left')
df_comp